# Julio Patti Pereira 
- 12/02/2025

# Introdução

Este documento é um complemento ao trabalho da disciplina de **Confecção de APIs** do curso de Especialização em Sistemas e Agentes Inteligentes da Universidade Federal de Goiás - **1ª Turma**. Embora seja relacionado ao trabalho principal, não corresponde ao corpo da solução desenvolvida (a API propriamente dita) e não possui o mesmo formalismo dos demais documentos externos à pasta "extra".

O estudo exploratório e o treinamento do modelo aqui apresentados não são os mais aprofundados possíveis. Ainda assim, a metodologia utilizada e algumas discussões interessantes serão definidas e exploradas.

### Contexto do Dataset

A escolha deste dataset foi motivada por dois principais fatores:

1. **Limitações quanto ao uso de dados reais**: Não obtive autorização da minha empresa para utilizar um dataset real de clientes. No entanto, este trabalho oferece a oportunidade de produzir uma API nos moldes que minha profissão exige. Ao utilizar um modelo treinado de exemplo, adaptado à confecção nos padrões do meu trabalho, posso facilitar o trabalho da equipe que realiza o deploy de soluções, sem a responsabilidade de adequar configurações específicas realizadas pelos times de sustentação, dos quais não faço parte.

2. **Interesse pessoal no dataset**: Conheci este dataset há alguns anos por meio de um tutorial de otimização bayesiana do **Mario Filho** (profissional conceituado e premiado em competições do Kaggle). Na ocasião, identifiquei um problema na abordagem oferecida no tutorial: a existência de muitas amostras duplicadas, o que pode ocasionar vazamento de dados de teste no treinamento quando não tratado adequadamente (e comumente é esquecido). Assim, a utilização deste dataset também serve para realizar experimentos básicos e sanar dúvidas pessoais, mesmo que com uma abordagem não muito profunda.

   - O link do tutorial do Mario Filho pode ser acessado em: [Tutorial de Otimização Bayesiana](https://www.youtube.com/watch?v=WhnkeasZNHI)

3. **Descrição das Colunas** :A seguir, apresenta-se a lista das variáveis presentes no dataset:

   - 1. **fixed acidity**: acidez fixa
   - 2. **volatile acidity**: acidez volátil
   - 3. **citric acid**: ácido cítrico
   - 4. **residual sugar**: açúcar residual
   - 5. **chlorides**: cloretos
   - 6. **free sulfur dioxide**: dióxido de enxofre livre
   - 7. **total sulfur dioxide**: dióxido de enxofre total
   - 8. **density**: densidade
   - 9. **pH**: potencial hidrogeniônico
   - 10. **sulphates**: sulfatos
   - 11. **alcohol**: teor alcoólico

   - **Variável de saída (com base em dados sensoriais):** 12. **quality**: qualidade (pontuação entre 0 e 10)

### Créditos

- O dataset está disponível no Kaggle: [Red Wine Quality - Cortez et al. (2009)](https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009)
- Corresponde à pesquisa de **Cortez et al., 2009**:
  - P. Cortez, A. Cerdeira, F. Almeida, T. Matos e J. Reis. *Modeling wine preferences by data mining from physicochemical properties*. **Decision Support Systems**, Elsevier, 47(4):547-553, 2009.

### Considerações Adicionais

Existem particularidades do dataset que não serão exploradas, como o desbalanceamento entre classes.

- Este aspecto pode fornecer estudos interessantes, uma vez que existem muitas críticas aos métodos tradicionais de tratamento, como o *undersampling* e, principalmente, o *oversampling* utilizando SMOTE.
- Contudo, neste trabalho, esse problema será tratado apenas com a utilização de *splits* estratificados de dados, para preservar a distribuição, e com pequenos ajustes, como o balanceamento de pesos nos modelos.
- **Não é o escopo deste trabalho** realizar um processo exaustivo de treinamento e exploração buscando a melhor solução possível.
- A descrição completa do dataset pode ser encontrada no link fornecido anteriormente, assim como diversos estudos a respeito dele.


# Metodologia e Justificativas

Inicialmente, realizou-se um tratamento nos dados para evitar casos de duplicidade e, consequentemente, vazamento de informações entre os conjuntos de treinamento e teste.
- Este procedimento, embora possa comprometer um pouco a distribuição real de dados, é mais indicado quando se quer medir a capacidade de generalização do modelo.

A separação dos dados foi efetuada destinando **20% das amostras** para simular o ambiente de produção. Este conjunto é completamente isolado dos demais, e nenhuma tomada de decisão é realizada com base nele. Servirá, ao final, apenas para avaliar se a resposta está de acordo com a esperada no treinamento.

- O conjunto de teste também será utilizado para fornecer amostras reais para testar a API desenvolvida (código principal).

A seleção do melhor modelo e a escolha dos hiperparâmetros foram realizadas com base no estudo do treinamento.

A etapa de treinamento foi conduzida utilizando **validação cruzada** em **80% das amostras**, divididas de forma **estratificada em 4 folds**. Inicialmente, um modelo padrão (modelo *std*) foi utilizado para medir a média, desvio padrão, mínimo e máximo dos resultados empregando os hiperparâmetros padrões, exceto pelo balanceamento dos pesos. Posteriormente, realizou-se o ajuste fino (*tuning*) dos hiperparâmetros com essa metodologia, visando verificar se o procedimento proporciona resultados superiores com a escolha adequada dos hiperparâmetros.

Os valores escolhidos para as amostragens, número de folds e faixa de hiperparâmetros foram determinados com base em experiências anteriores e nas referências mencionadas anteriormente, sem uma pesquisa científica ou estatística mais detalhada. Acredita-se que sejam suficientes para os propósitos deste estudo.

A **métrica de avaliação** selecionada, a princípio, foi a **ROC AUC** (*Receiver Operating Characteristic - Area Under the Curve*).

O alvo inicial (**quality**) foi binarizado em categorias como "bom" e "ruim", conforme sugerido nas referências. Contudo, a estratificação dos dados foi realizada com base na coluna original **quality**, como estratégia para maior homogeneização da distribuição antes de realizar qualquer transformação irreversível nos dados, como a própria binarização. Julgou-se essa estratégia conveniente para comparações futuras com abordagens diferentes, como em um tratamento multiclasse ou na utilização de modelos sequenciais.

O processo de validação cruzada foi realizado de modo mais manual do que o sugerido por bibliotecas populares, como o **scikit-learn**, pois, dessa forma, torna-se mais fácil realizar customizações futuras, como a aplicação de *undersampling*, entre outros.

- Essa prática também é conveniente em estudos com dados desbalanceados, uma vez que grande parte dos estudos disponíveis cometem alguns equívocos (segundo o consenso acadêmico) ao realizar, por exemplo, *oversampling* antes das divisões de treino, validação e teste. Essa prática leva, direta ou indiretamente, ao vazamento de dados. Embora não tenham sido utilizadas técnicas de amostragem neste estudo, caso isso seja abordado em trabalhos futuros, um processo de ajuste mais manual é conveniente para maiores customizações nos fluxos de dados.

**Algumas referências sobre problemas comuns com as estratégias de *oversampling*:**

- [How to do cross-validation when upsampling data](https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html)
- [Balanceamento de classes: cuidado, você pode estar fazendo errado](https://medium.com/lets-data/balanceamento-de-classes-cuidado-voc%C3%AA-pode-estar-fazendo-errado-b94188f1a37e)

Dentre as referências, há sugestões para utilizar soluções com a biblioteca **imbalanced-learn** ao invés do **scikit-learn**. Porém, em estudos próprios, já foram identificados dificuldades de ajustes com tais abordagens, sendo que grande parte da academia atualmente questiona os métodos de sampleamento tradicionais.


# Imports de bibliotecas

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from lightgbm import LGBMClassifier
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
import pickle

# Leitura de dados (raw)

In [2]:
path_data = '../data/raw_data.csv'

df = pd.read_csv(path_data)
raw_shape = df.shape
print(f'RAW SHAPE: {raw_shape}')
df

RAW SHAPE: (1599, 12)


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
# Substituindo espaços vazios nos nomes de colunas:
df.columns = [col.replace(' ', '_') for col in df.columns]
df.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


# Duplicados consistentes
- se os valores das features e da qualidade entre amostras forem os mesmos, as avaliações são consistentes, porém não agregam valor a um modelo de ML. 
- O split de dados duplicados pode resultar no vazamento de informações e prejudicar a avaliação de desempenho real

# Duplicados inconsistentes
- Seriam casos em que as features são suplicadas, porém a qualidade possui valores distintos
- O "keep=False", nesse caso, é interessante para verificar os casos. 
- Contudo, como será visto na segunda celula a seguir, casos assim não ocorreram

In [4]:
df.drop_duplicates(inplace=True)

new_shape = df.shape
print(f'{raw_shape[0] - new_shape[0]} amostras redundantes excluidas!')

240 amostras redundantes excluidas!


In [5]:
# Amostras inconsistentes (features vs labels)
inc_condiction = df.drop(columns=['quality']).duplicated(keep=False)

# Caso exista serão excluidos
df = df[~inc_condiction].reset_index(drop=True)

print(f'{inc_condiction.sum()} amostras inconsistentes!')

0 amostras inconsistentes!


# Distribuição por qualidade:

In [6]:
def dist_quality(df, column='quality'):
    count = df[column].value_counts()
    prop = df[column].value_counts(normalize=True)
    return pd.concat([count, prop], axis=1)

dist_quality(df)

,count,proportion
quality,,
5,577,0.424577
6,535,0.393672
7,167,0.122884
4,53,0.038999
8,17,0.012509
3,10,0.007358


# Stratified Split
- Mesmo que vamos binarizar o problema, acredito que esta seja a abordagem mais adequada para separação mais homogênea dos dados

In [7]:
sss = StratifiedShuffleSplit(n_splits=2, test_size=0.2, random_state=2025)
for train_index, test_index in sss.split(df, df['quality']):
    df_train, df_test = df.loc[train_index], df.loc[test_index]
    
df_train.shape[0]/df.shape[0], df_test.shape[0]/df.shape[0] 

(0.7998528329654158, 0.20014716703458427)

In [8]:
# Proporções de qualidades por subsets
view = pd.DataFrame()
for dset, dataframe in {'test': df_test, 'train': df_train, 'total': df}.items():
    view = pd.concat([view, dist_quality(dataframe).add_suffix('_'+dset)], axis=1)
view.loc['Total'] = view.sum()
view

,count_test,proportion_test,count_train,proportion_train,count_total,proportion_total
quality,,,,,,
5,116.0,0.426471,461.0,0.424103,577.0,0.424577
6,107.0,0.393382,428.0,0.393744,535.0,0.393672
7,33.0,0.121324,134.0,0.123275,167.0,0.122884
4,11.0,0.040441,42.0,0.038638,53.0,0.038999
8,3.0,0.011029,14.0,0.012879,17.0,0.012509
3,2.0,0.007353,8.0,0.007360,10.0,0.007358
Total,272.0,1.000000,1087.0,1.000000,1359.0,1.000000


# Save splits
- É uma boa pratica deixar o conjunto que ira simular a produção (df_test) separado do conjunto de treinamento
- o conjunto de produção não será utilizado para qualquer tomada de decisão neste treinamento
- o conjunto de produção poderá ser utilizado para testes do produto final (API)

In [9]:
df_train.to_csv('../data/df_train.csv', index=False)
df_test.to_csv('../data/df_test.csv', index=False)

# Prevenção de vazamento
del df_test, df_train, df

# Import train data

In [10]:
df_train = pd.read_csv('../data/df_train.csv')
df_train.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality
count,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000,1087.000000
mean,8.264305,0.531463,0.267056,2.513937,0.087316,15.858786,46.559798,0.996671,3.313017,0.654802,10.432475,5.625575
std,1.707009,0.186630,0.193222,1.331599,0.046865,10.534171,33.126142,0.001839,0.154486,0.160925,1.073139,0.824693
min,4.600000,0.120000,0.000000,1.200000,0.012000,1.000000,6.000000,0.990200,2.880000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.069000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,38.000000,0.996680,3.310000,0.620000,10.100000,6.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997800,3.400000,0.720000,11.100000,6.000000
max,15.900000,1.580000,0.760000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,1.950000,14.900000,8.000000


# Binarização do problema
- Será considerado que o vinho é "bom" se ele tiver uma nota igual ou superior a 7
- O problema sera reduzido ao par 0, 1, sendo 1 a classe dos vinhos "bons"

In [11]:
df_train['bin_quality'] = df_train['quality'].astype(int)>=7
df_train['bin_quality'] = df_train['bin_quality'].astype(int)

feature_columns = df_train.drop(columns=['quality', 'bin_quality']).columns.tolist()
dist_quality(df_train, column='bin_quality')

,count,proportion
bin_quality,,
0,939,0.863845
1,148,0.136155


In [12]:
dist_quality(df_train[df_train['bin_quality']==1], column='quality')

,count,proportion
quality,,
7,134,0.905405
8,14,0.094595


In [13]:
dist_quality(df_train[df_train['bin_quality']==0], column='quality')

,count,proportion
quality,,
5,461,0.490948
6,428,0.455804
4,42,0.044728
3,8,0.008520


# Std solution
- A validação cruzada será realizada com split estratificado na "quality" propositadamente para maior homogeneidade dos dados
- Ela também será realizada de modo "menos automatico", pois caso algum futuro estudo de sampleamento no treinamento seja realizado, a estratégia pode ser mais bem customizado sem as bibliotecas que realizam esse procedimento de modo automatico

In [14]:
def calcular_estatisticas(lista, output=True):
    
    mean = round(np.mean(lista) * 100, 1)
    std = round(np.std(lista) * 100, 1)
    min = round(np.min(lista) * 100, 1)
    max = round(np.max(lista) * 100, 1)
    
    if output:
        output=f"Média da validação cruzada (std): {mean}% ({std}%)\n(Min/Máx): ({min}%/{max}%)"
        print(output)
    
    return mean, std, min, max

In [15]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=2025)
auc = []
for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
    df_train_fold, df_val_fold = df_train.loc[train_index_fold], df_train.loc[val_index_fold]
    
    X_fold = df_train_fold[feature_columns]
    y_fold = df_train_fold['bin_quality']
    X_val = df_val_fold[feature_columns]
    y_val = df_val_fold['bin_quality']
    
    model = LGBMClassifier(class_weight='balanced', verbose=-1)
    model.fit(X_fold, y_fold)
    # y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:,1]
    auc.append(roc_auc_score(y_val, y_val_proba))
    
# Calcular média, desvio padrão, mínimo e máximo
print('AUC')
mean, std, min, max = calcular_estatisticas(auc)

AUC
Média da validação cruzada (std): 86.4% (2.4%)
(Min/Máx): (83.6%/89.0%)


# Otimização de hiperparâmetros

In [16]:
space = [Real(low=1e-4, high=3e-1, prior='log-uniform'),   # learning_rate     = params[0]
         Integer(low=2, high=128),                         # num_leaves        = params[1]
         Integer(low=1, high=200),                         # min_child_samples = params[2]
         Integer(low=100, high=500),                       # max_bin           = params[3]
         Real(low=0.05, high=1.0),                         # subsample         = params[4]
         Real(low=0.1, high=1.0),                          # colsample_bytree  = params[5]
         Integer(low=100, high=300)                        # n_estimators      = params[6]
         ]


def get_model(params):

    model = LGBMClassifier(
        learning_rate     = params[0],
        num_leaves        = params[1],
        min_child_samples = params[2],
        max_bin           = params[3],
        subsample         = params[4],
        colsample_bytree  = params[5],
        n_estimators      = params[6],
        subsample_freq    = 1,
        class_weight      = 'balanced',
        random_state      = 2025
    )
    
    return model


def objective_minimize(params):
    # print(params)
    auc = []
    for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
        df_train_fold, df_val_fold = df_train.loc[train_index_fold], df_train.loc[val_index_fold]
        X_fold = df_train_fold[feature_columns]
        y_fold = df_train_fold['bin_quality']
        X_val = df_val_fold[feature_columns]
        y_val = df_val_fold['bin_quality']
        
        model = get_model(params)
        model.fit(X_fold, y_fold)
        # y_val_pred = model.predict(X_val)
        y_val_proba = model.predict_proba(X_val)[:,1]
        auc.append(roc_auc_score(y_val, y_val_proba))
    metric = float(np.mean(auc))
    
    return - metric

In [17]:
# Otimização de (hiper)parametros
result = gp_minimize(objective_minimize, space, random_state=2025, n_calls=100, n_random_starts=30)

# Melhores parâmetros
best_params = result.x
print(best_params)

[0.003276827920829083, np.int64(126), np.int64(8), np.int64(279), 0.25075152901548947, 0.506636140858595, np.int64(195)]


# Escolha e avaliação do modelo
- Primeiramente deve-se avaliar o desempenho médio e desvios para decidir se a configuração utilizada produz a consistencia desejada
- Verificar se há melhoria em comparação a outras estratégias
- Por fim, deve-se especificar o modelo. Dentre os processos de escolha, é comum:
    - Dado os melhores parâmetros, utilizar o modelo de melhor desempenho nas split folds da validação cruzada
    - Realizar o "Reffit" do modelo, isto é, com os melhores hiperparâmetros treinar um novo modelo com a totalidade dos dados de test_index
- Como se tem uma amostragem reduzida da classe minoritária, optou-se pela segunda abordagem, visto que nela um número maior de dados é utilizado em treinamento
    - Mesmo assim, a validação cruzada com os "melhores parâmetros" será realizada para a comparação com a avaliação em modo padrão, realizada anteriormente.

In [18]:
# Validação cruzada
auc = []
for train_index_fold, val_index_fold in skf.split(df_train, df_train['quality']):
    df_train_fold, df_val_fold = df_train.loc[train_index_fold], df_train.loc[val_index_fold]
    X_fold = df_train_fold[feature_columns]
    y_fold = df_train_fold['bin_quality']
    X_val = df_val_fold[feature_columns]
    y_val = df_val_fold['bin_quality']
    
    model = get_model(best_params)
    model.fit(X_fold, y_fold)
    # y_val_pred = model.predict(X_val)
    y_val_proba = model.predict_proba(X_val)[:,1]
    auc.append(roc_auc_score(y_val, y_val_proba))
    
# Calcular média, desvio padrão, mínimo e máximo
print('AUC')
mean, std, min, max = calcular_estatisticas(auc)

AUC
Média da validação cruzada (std): 88.3% (2.4%)
(Min/Máx): (84.2%/90.3%)


In [19]:
# Modelo final
X_train = df_train[feature_columns]
y_train = df_train['bin_quality']

model = get_model(best_params)
model.fit(X_train, y_train)

LGBMClassifier(class_weight='balanced', colsample_bytree=0.506636140858595,
               learning_rate=0.003276827920829083, max_bin=np.int64(279),
               min_child_samples=np.int64(8), n_estimators=np.int64(195),
               num_leaves=np.int64(126), random_state=2025,
               subsample=0.25075152901548947, subsample_freq=1)

# Simulação de desempenho em produção/blind set
- Metricas gerais
Obs: dada a métrica de otimização, a simulação teve um desempenho que muito condiz com a avaliação, isto é, não só perto da média como dentro de um pequeno desvio.
- Avaliação: Media 88,3% e 2,4% std (AUC)
- Blind test: 87,4% (AUC)

In [20]:
df_test = pd.read_csv('../data/df_test.csv')
df_test['bin_quality'] = (df_test['quality']>=7).astype(int)
df_test.head(2)

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,bin_quality
0,6.8,0.61,0.20,1.8,0.077,11.0,65.0,0.99710,3.54,0.58,9.3,5,0
1,11.3,0.36,0.66,2.4,0.123,3.0,8.0,0.99642,3.20,0.53,11.9,6,0


In [21]:
X_test = df_test.drop(columns=['quality','bin_quality'])
y_test = df_test['bin_quality']

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

auc = roc_auc_score(y_test, y_pred_proba)
report = classification_report(y_test, y_pred)

print(f'AUC: {round(100*auc,1)}%\n\n{report}')

AUC: 87.4%

              precision    recall  f1-score   support

           0       0.94      0.89      0.91       236
           1       0.46      0.61      0.52        36

    accuracy                           0.85       272
   macro avg       0.70      0.75      0.72       272
weighted avg       0.87      0.85      0.86       272



# Avaliação de Subsets
- visto que a qualidade dos vinhos bons tinham notas 7 ou 8, vamos visualizar o resultado do modelo binário sobre esses subsets
    - Como se tivessemos mais interesse, por um momento, nos vinhos mais bem abvaliados
        - Podemos perceber que a qualidade 7 teve bastante confusão com as qualidades inferiores
        - A amostragem da qualidade 8 é muito pequena para se tomar alguma conclusão solida, mesmo que o acerto tenha sido 100%
            - porem, acredito que o dipo de estratificação mais homogenizado realizado pode ter favorecido o acerto

In [22]:
df_test['proba'] = y_pred_proba
df_test['pred'] = y_pred

df_test_quality_7 = df_test[df_test['quality']==7].copy()
dist_quality(df_test_quality_7, column='pred')

,count,proportion
pred,,
1,19,0.575758
0,14,0.424242


In [23]:
df_test_quality_8 = df_test[df_test['quality']==8].copy()
dist_quality(df_test_quality_8, column='pred')

,count,proportion
pred,,
1,3,1.0


# Save Model

In [24]:
path_model = '../ml_models/modelo_01.pkl'
with open(path_model, 'wb') as arquivo:
    pickle.dump(model, arquivo)

# Test loaded model

In [25]:
with open(path_model, 'rb') as arquivo:
    model = pickle.load(arquivo)

y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, y_pred_proba)
report = classification_report(y_test, y_pred)

print(f'AUC: {round(100*auc,1)}%\n\n{report}')

AUC: 87.4%

              precision    recall  f1-score   support

           0       0.94      0.89      0.91       236
           1       0.46      0.61      0.52        36

    accuracy                           0.85       272
   macro avg       0.70      0.75      0.72       272
weighted avg       0.87      0.85      0.86       272



# Julio Patti Pereira 
- 12/02/2025